Problme statement:
Classify the given medical text into below 5 categories:


1.   Digestive system diseases
2.   cardiovascular diseases
3.   Neoplasms
4.   nervous system disease
5.   General pathological conditions


Input format

{
    
    "medical_text":"${text} "

}

output format

{
    
    "Category": "4",

    "Description": "nervous system disease",

    "medical text": "${text} "
}





#### Import libraries

In [1]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [2]:
import pandas as pd

In [3]:
# uninstall
!pip uninstall -y wandb

# download
!pip install transformers
!pip install simplet5

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.7/527.7 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.4/952.4 kB 70.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 70.8 MB/s eta 0:00:00
  Created wheel for simplet5: filename=simplet5-0.1.4-py3-none-any.whl size=6853 sha256=6c938fcce7cdadb503623cbeddf8bb4a0396acad881bf530f4f7b7e9183ef1c9
  Stored in directory: /root/.cache/pip/wheels/b4/7d/af/743765400878438a7593f13f89fdf4004dcde0f2a8e6cb6684
Successfully built simplet5
  Attempting uninstall: setuptools
    Found existing installation: setuptools 67.7.2
    Uninstalling setuptools-67.7.2:
      Successfully uninstalled setuptools-67.7.2
  Attempti

In [4]:
import re
import json
import torch

import random
import pandas as pd
from tqdm import tqdm
from simplet5 import SimpleT5
from torch.utils.data import Dataset

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


#### Data formatting and understanding

In [5]:
# Data load function
def load_dataset():
  train_df = pd.read_csv("/content/train.dat", sep="\t",index_col=False)
  train_df.columns=["target_text","source_text"]
  train_df['source_text'] = train_df['source_text'].astype('str')
  train_df['target_text'] = train_df['target_text'].astype('str')
  train_df.reset_index(drop=True,inplace=True)

  test_df=pd.read_csv("/content/train.dat", sep="\t",index_col=False)
  test_df.columns=["target_text","source_text"]
  test_df['target_text'] = test_df['target_text'].astype('str')
  test_df['source_text'] = test_df['source_text'].astype('str')
  test_df.reset_index(drop=True,inplace=True)

  return train_df,test_df


In [6]:
train_df,test_df=load_dataset()

In [ ]:
train_df.head()

In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14437 entries, 0 to 14436
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   target_text  14437 non-null  object
 1   source_text  14437 non-null  object
dtypes: object(2)
memory usage: 225.7+ KB


In [8]:
test_df.head()

,target_text,source_text
0,5,Renal abscess in children. Three cases of rena...
1,2,Hyperplastic polyps seen at sigmoidoscopy are ...
2,5,Subclavian artery to innominate vein fistula a...
3,4,Effect of local inhibition of gamma-aminobutyr...
4,1,Infection during chronic epidural catheterizat...


#### Model training and validation on test data

In [9]:
from glob import glob

for trial_no in range(1):
    # create data
    train_df, test_df = load_dataset()
    # load model
    model = SimpleT5()
    model.from_pretrained(model_type="t5", model_name="t5-base")
    # train model
    model.train(train_df=train_df,
                eval_df=test_df,
                source_max_token_len=300,
                target_max_token_len=200,
                batch_size=8,
                max_epochs=2,
                outputdir = "outputs",
                use_gpu=True
               )
    # fetch the path to last model
    last_epoch_model = None
    for file in glob("./outputs/*"):
        if 'epoch-1' in file:
            last_epoch_model = file
    # load the last model
    model.load_model("t5", last_epoch_model, use_gpu=True)
    # test and save
    # for each test data perform prediction
    predictions = []
    for index, row in test_df.iterrows():
        prediction = model.predict(row['source_text'])[0]
        predictions.append(prediction)
    df = test_df.copy()
    df['predicted'] = predictions
    df['original'] = df['target_text']
    print(f1_score(df['original'], df['predicted'], average='macro'))
    df.to_csv(f"result_run_{trial_no}.csv", index=False)
    # clean the output
    #!rm -rf ./outputs

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


...

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (519 > 512). Running this sequence through the model will result in indexing errors


0.6987286014144732


### Note :Above finetunning showing F1 of 0.698

In [10]:
!zip -r /content/file.zip /content/outputs

  adding: content/outputs/ (stored 0%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/ (stored 0%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/tokenizer.json (deflated 74%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/special_tokens_map.json (deflated 83%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/config.json (deflated 63%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/spiece.model (deflated 48%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/tokenizer_config.json (deflated 80%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/pytorch_model.bin (deflated 9%)
  adding: content/outputs/simplet5-epoch-0-train-loss-0.5418-val-loss-0.3923/ (stored 0%)
  adding: content/outputs/simplet5-epoch-0-train-loss-0.5418-val-loss-0.3923/tokenizer.json (deflated 74%)
  adding: content/o

In [12]:
!zip -r /content/file_01.zip /content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532

  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/ (stored 0%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/tokenizer.json (deflated 74%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/special_tokens_map.json (deflated 83%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/config.json (deflated 63%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/spiece.model (deflated 48%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/tokenizer_config.json (deflated 80%)
  adding: content/outputs/simplet5-epoch-1-train-loss-0.4069-val-loss-0.3532/pytorch_model.bin (deflated 9%)


In [13]:
from google.colab import files
files.download("/content/file_01.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>